#Set up environment

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Voice_coversion

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Voice_coversion
True
Tesla K80


In [2]:
from model import Generator
import torch
import torch.nn.functional as F
import time
import datetime
import data_loader

In [3]:
# hyper_parameters

# model 
dim_neck = 16
dim_emb = 256
dim_pre = 512
freq = 16

# training
lambda_cd = 1
learning_rate = 0.0001
batch_size = 2
num_iter = 1000000
len_crop = 128

# log
log_step = 10

In [4]:
data_path = 'spmel'

loader = data_loader.get_loader(data_path, batch_size, len_crop)

Finished loading the dataset...


In [5]:
from torch.backends import cudnn

# for fast training
cudnn.benchmark = True

In [6]:
generator = Generator(dim_neck, dim_emb, dim_pre, freq)
optimizer = torch.optim.Adam(generator.parameters(), learning_rate)

generator = generator.to(torch.device("cuda"))

In [ ]:
# the main training loop
for i in range(num_iter):
  # data preparation
  try:
    x_real, emb_org = next(data_iter)
  except:
    data_iter = iter(loader)
    x_real, emb_org = next(data_iter)
            
            
  x_real = x_real.to(torch.device("cuda")) 
  emb_org = emb_org.to(torch.device("cuda"))

  # forward
  x_identic, x_identic_psnt, code_real = generator(x_real, emb_org, emb_org)

  # calculate the loss
  #     Decoder loss
  g_loss_id = F.mse_loss(x_real, x_identic)   
  g_loss_id_psnt = F.mse_loss(x_real, x_identic_psnt)

  #     Reconstruction loss
  code_reconst = generator(x_identic_psnt, emb_org, None)
  g_loss_cd = F.l1_loss(code_real, code_reconst)

  #     Total loss
  loss = g_loss_id + g_loss_id_psnt + lambda_cd*g_loss_cd

  # back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % log_step == 0: 
    print('Iteration: {}/{}, Loss: {}'.format(i+1, num_iter, loss))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Using a target size (torch.Size([2, 1, 128, 80])) that is different to the input size (torch.Size([2, 128, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Using a target size (torch.Size([2, 1, 128, 80])) that is different to the input size (torch.Size([2, 128, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Iteration: 1/1000000, Loss: 1.4535130262374878
Iteration: 11/1000000, Loss: 1.2556605339050293
Iteration: 21/1000000, Loss: 1.0529085397720337
Iteration: 31/1000000, Loss: 0.8814471960067749
Iteration: 41/1000000, Loss: 0.724055826663971
Iteration: 51/1000000, Loss: 0.6799180507659912
Iteration: 61/1000000, Loss: 0.6834350228309631
Iteration: 71/1000000, Loss: 0.6419917345046997
Iteration: 81/1000000, Loss: 0.6212701797485352
Iteration: 91/1000000, Loss: 0.6482723355293274
Iteration: 101/1000000, Loss: 0.6190829873085022
Iteration: 111/1000000, Loss: 0.6008784174919128
Iteration: 121/1000000, Loss: 0.6054587364196777
Iteration: 131/1000000, Loss: 0.5858967304229736
Iteration: 141/1000000, Loss: 0.5791098475456238
Iteration: 151/1000000, Loss: 0.597093403339386
Iteration: 161/1000000, Loss: 0.5815885663032532
Iteration: 171/1000000, Loss: 0.5982595682144165
Iteration: 181/1000000, Loss: 0.5868270993232727
Iteration: 191/1000000, Loss: 0.5905157327651978
Iteration: 201/1000000, Loss: 0.5